## Circuit graph and symbolic Hamiltonian 

### Circuit graph 

In [ ]:
zp_yaml = """# zero-pi
branches:
- [JJ, 1,2, EJ = 10nA, 200pF]
- [JJ, 3,4, EJ, 20GHz]
- [L, 2,3, 10uH]
- [L, 4,1, 0.008]
- [C, 1,3, 0.02]
- [C, 2,4, 0.02]
"""

The example above illustrates most of the syntax rules to be followed. Each branch is represented by 

```<branch-type>, <node_1>, <node_2>, <param-1> [, <param-2>]```

### Branch types and parameters:

- `C`:  branch parameter is the charging energy $E_C = \frac{e^2}{2C}$
- `L`:  branch parameter is the inductive energy $E_L = \frac{\Phi_0^2}{(2\pi)^2 L}$
- `JJ`: branch parameters are the Josephson energy $E_J = \frac{\hbar I_c}{2\pi} = L_J I_c^2$ and junction charging energy $E_{CJ}$

*Example:* `- [C, 1, 3, 0.02]` is a capacitance connecting nodes 1 and node 3, with charging energy 0.02 GHz.



### Inputing branch parameters using physical units

The branch parameters can now be entered using physical units prepended by a multiplier. Any branch parameter can be gievn as energy with the units: $Hz$, $J$ or $eV$. Though, each of the branch parameter can be entered with its own unit as well, as listed in the following table.

| Parameter   | Units       |
| ----------- | ----------- |
| C           | F           |
| L           | H           |
| EJ          | A           |

The above units can be prepended with the following multipliers:

| Multiplier  | Value       |
| ----------- | ----------- |
| f           | $10^{-15}$  |
| p           | $10^{-12}$  |
| n           | $10^{-9}$   |
| u           | $10^{-6}$   |
| m           | $10^{-3}$   |
| k           | $10^{3}$    |
| M           | $10^{6}$    |
| G           | $10^{9}$    |
| T           | $10^{12}$   |

For example the inductance of a branch can be given as `1uH`, and Josephson junction energy can be given as `1uA` or `40eV`. If no units are provided the default unit is $GHz$.

### Symbolic vs. numerical branch parameters:

- Branch parameters can be provided as float values, using the energy units set globally (default: GHz)
- A symbol name can be specified, but must be introduced along with a value (e.g., `EJ = 10`). Note that the same symbol can afterwards be used to set any other compatible branch parameter. For instance, in the YAML description `zp_yaml`, the same parameter $EJ$ is used for the Josephson energies of the junctions connecting nodes $(1, 2)$ and $(3, 4).$
- Symbolic output will generally maintain symbolic parameter names. However, for charging energies of systems with more than three nodes, numerical values are the default, since symbolic inversion of larger capacitance matrices is generally very slow and yields unwieldy expressions.

<div class="alert alert-info">
    
Accessing circuit parameters
    
- Only circuit parameters with an associated name (as in `EJ = 10`) are made accessible as attributes of the class instance: `<Circuit instance>.EJ`. 
    
- Parameter sweeps (e.g., via `plot_evals_vs_paramvals()`) can only be performed for those circuit parameters that have been provided with a symbol name.
</div>

### Ground node:

- The user can choose to include an explicit ground node by giving this node the label `0`. Such ground node serves as a reference node with generalized flux always $\phi_0 = 0$.

### Initiate the `Circuit` object

Using the yaml input, create a `Circuit` object:

In [ ]:
zero_pi = scq.Circuit(zp_yaml, from_file=False)

### Define Circuit with symbolic hamiltonian

A `Circuit` instance can also be initated with a symbolic Hamiltonian. The Hamiltonian can be written using `sympy.parse_expr`. The returned symbolic expression can be used with the Circuit module. 

The expression should adhere to the following rules:

- Each extended degree of freedom uses the conjugate variables $\theta_i$ and $Q_i$.
- Each periodic degree of freedom uses the conjugate variables $\theta_i$ and $n_i$.
    - if $\theta_i$ is periodic, the variable should not appear outside a trigonometric function. 
- The offset charge is represented by $n_{gi}$ and external flux by $\Phi_i$. The external flux variables are in the units of flux quantum $\Phi_0$.
- Any symbolic parameters used, should be defined with its initial value in the dictionary `symbolic_params_dict`.

Here is an example demonstrating the use of symbolic Hamiltonian:

In [1]:
import sympy as sm
import scqubits as scq

sym_hamiltonian = sm.parse_expr(
    "0.25*θ3**2 + 2.0*Q3**2 + 0.790*Q2**2 + 0.45*θ2**2 + 7.674*n1**2 + 7.674*ng1**2 - EJ*cos(θ1) + 0.5*θ2*θ3 + 1.395*Q2*n1 + 1.395*Q2*ng1 + 15.348*n1*ng1"
)
circ = scq.Circuit(
    symbolic_hamiltonian=sym_hamiltonian,
    symbolic_param_dict={"EJ": 10},
    ext_basis="harmonic",
)
circ

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Circuit_1